# **Setup the development framework such that changes in py files auto load into this file**
- You can ignore this if your interest is only in the business logic and design. 
- The steps in this section are only environment set related

### **Make sure py file change are loaded automatically**

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### **Import basic package required for this assignment**

In [14]:
import os
import pandas as pd
import db.constants as C


### **Make sure right sources are getting picked**

In [15]:
from dao.base_transformer import BaseDBTransformer

### **Make sure the latest class are getting loaded.**

In [16]:
# 1. Import the modules first (so Python knows them)
import importlib
import service.product_transformer
import service.cart_transformer
import service.purchase_transformer
import service.discount_transformer
import db.constants
import db.dbsql

# 2. Reload them (useful after edits)
importlib.reload(service.product_transformer)
importlib.reload(service.cart_transformer)
importlib.reload(service.purchase_transformer)
importlib.reload(service.discount_transformer)
importlib.reload(db.constants)

# 3. Import the classes fresh from the reloaded modules
from service.product_transformer import ProductTransformer
from service.cart_transformer import CartTransformer
from service.purchase_transformer import PurchaseTransformer
from service.discount_transformer import DiscountTransformer
import db.constants
import random

# **Transformers**
- This section is the core design of the project
- There are four transformers 
   - Products transformer: This is the database of the list of the products in EStore. The prodcuts.csv file for demo has 9 products. 
   - Cart transformer: This is user selected products added into the cart instance specific to the user. 
   - purchase transfromer: This is checkout functionaility of the cart. 
        - It has two kinds of purchases. With discount by applying discount code.
        - It also supports purchase without applying discount code. If no discount is applied it generates a ne discount code and adds to discounts for the subsequent purchases. 
   - Discount transformer: This handles the discounts codes after the purchase. 

   - For each these transforers there is .py  file and .csv file in the src directory. 
   - Each transfomer derives from the base transfomer class which manages the csv read/write for that transformer. 
   - The base transformer class also handles basis CRUD operation if they are at the level of the dataframe(entire row level operations) without having to deal with individual columns (as columns are specific to the transformer)

- The code is organized as a typical pyproject.toml file. 

## **Product Transformer - demonstration**

In [17]:
from db.dbsql import SessionLocal, Product, ProductClass

# Product.c.get('PRODUCT_CLASS_CODE')
Product.c.PRODUCT_CLASS_CODE

Column('PRODUCT_CLASS_CODE', INTEGER(), ForeignKey('product_class.PRODUCT_CLASS_CODE'), table=<product>, nullable=False)

In [18]:
pt = ProductTransformer()

res = pt.list_products_in_class_df(2050)

res

,PRODUCT_ID,PRODUCT_DESC,PRODUCT_CLASS_CODE,PRODUCT_PRICE,PRODUCT_QUANTITY_AVAIL,LEN,WIDTH,HEIGHT,WEIGHT
0,1,Sky LED 102 CM TV,2050,35000.0,30,905,750,700,15.00
1,2,Sams 192 L4 Single-door Refrigerator,2050,28000.0,15,1802,750,750,25.00
2,3,Jocky Speaker Music System HT32,2050,8900.0,19,908,300,300,5.00
3,21,Cybershot DWC-W325 Camera,2050,5300.0,5,100,55,40,0.05


In [19]:
pt.list_all_products()

,PRODUCT_ID,PRODUCT_DESC,PRODUCT_CLASS_CODE,PRODUCT_PRICE,PRODUCT_QUANTITY_AVAIL,LEN,WIDTH,HEIGHT,WEIGHT
0,1,Sky LED 102 CM TV,2050,35000.0,30,905,750,700,15.000
1,2,Sams 192 L4 Single-door Refrigerator,2050,28000.0,15,1802,750,750,25.000
2,3,Jocky Speaker Music System HT32,2050,8900.0,19,908,300,300,5.000
3,4,Cricket Set for Boys,2051,4500.0,10,890,300,200,18.000
4,5,Infant Sleepwear Blue,2052,250.0,50,596,300,100,0.250
5,6,Barbie Fab Gown Doll,2051,1000.0,20,305,150,75,0.150
6,7,Remote Control Car,2051,2900.0,12,200,150,50,0.225
7,8,Doll House,2051,3000.0,12,600,455,375,0.900
8,9,Blue Jeans 34,2052,800.0,100,450,310,52,1.100
9,10,Blossoms Lehenga Choli set,2052,3000.0,100,600,315,54,0.250


## **Discount Transformer - demonstration**

In [20]:
dt = DiscountTransformer()
dt.list_eligible_codes(1)

,DISCOUNT_ID,ORDER_ID,CUSTOMER_ID,DISCOUNT_CODE,DISCOUNT_PERCENT,DISCOUNT_STATUS
0,10001_1,10001,1,NO_DISCOUNT,-1,0


## **Purchase Transformer - demonstration**

In [21]:
prt = PurchaseTransformer()
prt.list_orders_by_customer(2)

,ORDER_ID,CUSTOMER_ID,PAYMENT_MODE,SHIPPER_ID,DISCOUNT_ID
0,10002,2,Cash,50001,1000
1,10012,2,Credit Card,50004,1000
2,10037,2,Cash,50005,1000


## **Cart Transformer - demonstration**

In [22]:
ct = CartTransformer
ct.list_all()

,PRODUCT_ID,PRODUCT_QUANTITY,CUSTOMER_ID
0,205,3,1
1,240,2,2
2,28,4,3
3,34,4,3
4,39,4,3
5,17,4,3
6,1,4,3


# **Business logic - Helper functions**

In [27]:
prt = ProductTransformer()
products_to_buy = prt.getRandomProducts(5)
customer_id=3
products_to_buy


,PRODUCT_ID,PRODUCT_DESC,PRODUCT_CLASS_CODE,PRODUCT_PRICE,PRODUCT_QUANTITY_AVAIL,LEN,WIDTH,HEIGHT,WEIGHT
21,22,KitchAsst Siphon Coffee Maker 500 ml,2060,1790.0,10,150,100,200,1.200
11,12,Samsung Galaxy On6,2055,14000.0,20,120,70,18,0.650
12,13,Alchemist,2054,150.0,50,200,100,20,0.235
20,21,Cybershot DWC-W325 Camera,2050,5300.0,5,100,55,40,0.050
7,8,Doll House,2051,3000.0,12,600,455,375,0.900


In [28]:
ct = CartTransformer()
ct.addToCart(customer_id, products_to_buy, False)
items = ct.list_cust(customer_id)
items


,PRODUCT_ID,PRODUCT_QUANTITY,CUSTOMER_ID
0,22,1,3
1,12,1,3
2,13,1,3
3,21,1,3
4,8,1,3


In [ ]:
def purchase_cart_items(cust_id, shipper_id=50001, discount_id='00000', pay_mode='Credit Card', debug=False):
     
    # First create order_header (order_id)
    ordh_dict = {}
    ordh_dict[C.ordid] = 0
    ordh_dict[C.custid] = cust_id
    ordh_dict[C.pmode] = pay_mode
    ordh_dict[C.shipid] = shipper_id
    ordh_dict[C.did] = discount_id

    res = BaseDBTransformer.insert(C.orders,ordh_dict)
    print(res)

    cust_cart = BaseDBTransformer.read(C.cart, **{C.custid: cust_id})
    cust_cart

In [30]:
purchase_cart_items(customer_id)

AttributeError: module 'db.constants' has no attribute 'shipid'

# **Admin APIs**

In [14]:
def getPurchases():
    prt = PurchaseTransformer()
    print(prt.list_all_orders())

def getDiscounts():
    dt = DiscountTransformer()
    print(dt.list_all_codes())

getPurchases()
getDiscounts()

    ORDER_ID  CUSTOMER_ID PAYMENT_MODE  SHIPPER_ID DISCOUNT_ID
0      10001            1  Credit Card       50001        1000
1      10002            2         Cash       50001        1000
2      10003            5         Cash       50001        1000
3      10004            5  Credit Card       50001        1000
4      10005            5  Credit Card       50002        1000
..       ...          ...          ...         ...         ...
65     10066           41                        0        1000
66     10067            6  Credit Card           0        1000
67     10068           51  Credit Card       50004        1000
68     10069           23  Net Banking       50002        1000
69     10070           10  Net Banking       50001        1000

[70 rows x 5 columns]
  DISCOUNT_ID  ORDER_ID  CUSTOMER_ID DISCOUNT_CODE  DISCOUNT_PERCENT  \
0     10001_1     10001            1   NO_DISCOUNT                -1   
1     10002_2     10002            2   NO_DISCOUNT                -1   

   D

# **Use cases 1 - Buy products**

# **Use cases 2 - Buy products with discount**